# Importing Required Libraries

In [69]:
import pandas as pd
import os

# Combining all the data in a single dataframe

In [70]:
cp=os.getcwd()
data=[]
for dir , _ , fp in os.walk(cp):
    for f in fp:
        if f.endswith('.csv') and f!='product_names.csv' and f!='sales_summary.csv':
            filepath=os.path.join(dir,f)
            df=pd.read_csv(filepath)
            data.append(df)
combined_data=pd.concat(data,ignore_index=True)
combined_data.head()

,Date,Store ID,Product ID,Quantity sold
0,2001-02-04,S1,P1,53
1,2001-02-07,S1,P2,95
2,2001-02-08,S1,P3,89
3,2001-02-15,S1,P4,49
4,2001-02-20,S1,P5,130


# Dropping data of invalid dates.

In [71]:
combined_data['Date']=pd.to_datetime(combined_data['Date'],errors='coerce')

# Handling Error

In [72]:
rows_to_drop=[]
for i,r in combined_data.iterrows():
    quantity=r['Quantity sold']
    if not isinstance(quantity,list):
        quantity=[quantity]
    try:
        for q in quantity:
            if q<0:
                raise ValueError("Quantity cannot be negative.")
            elif not isinstance(q,int):
                raise TypeError("Quantity must be an integer.")
    except ValueError as e:
        print(e)
        rows_to_drop.append(i)
    except TypeError as e:
        print(e)
        rows_to_drop.append(i)
combined_data=combined_data.drop(rows_to_drop).reset_index(drop=True)

# Calculating Total sales for each product across all stores and months

In [73]:
total=combined_data.groupby('Product ID')['Quantity sold'].sum().reset_index()
total.head()

,Product ID,Quantity sold
0,P1,3371
1,P2,3403
2,P3,3455
3,P4,3024
4,P5,3424


# Calculating top 5 best selling products in terms of sales

In [74]:
top5=total.sort_values('Quantity sold',ascending=False).head()
print("Top 5 best selling products: ")
for pid,quantity in zip(top5['Product ID'],top5['Quantity sold']):
    print(f"{pid} : {quantity}")

Top 5 best selling products: 
P3 : 3455
P5 : 3424
P2 : 3403
P1 : 3371
P4 : 3024


# Calculating the average quantity sold per month

In [75]:
combined_data['Month']=combined_data['Date'].dt.month
avg=combined_data.groupby(['Product ID','Month'])['Quantity sold'].sum().reset_index()
final_avg=avg.groupby('Product ID').agg(Average_Quantity_Sold=('Quantity sold','mean')).reset_index()

# Creating a new csv file "Sales_summary.csv"

In [76]:
pdf=pd.read_csv('product_names.csv')
m1=pd.merge(pdf,total,on='Product ID',how='inner')
m2=pd.merge(m1,final_avg,on='Product ID',how='inner')
m2.to_csv('sales_summary.csv',index=False)